In [1]:
import pandas as pd
import numpy as np
import ast

In [4]:
reprisk_incidents = pd.read_csv(
    'data/wrds_reprisk_incidents_ids_material_short.csv')  # Calculation in "CreateDataset" of "ESGmateriality" project
reprisk_incidents = reprisk_incidents[
    ['gvkey', 'isin', 'cusip', 'reprisk_id', 'story_id', 'adopter', 'sic', 'SICS Codified Industry ',
     'Codified SICS Sector ', 'severity', 'reach', 'novelty', 'incident_list', 'incident_date', 'cusip_cusip_tic','car_5', 'car_1', 'material_flag']]

In [7]:
# Create Year, YearMonth, YearQuarter columns
reprisk_incidents['incident_date'] = pd.to_datetime(reprisk_incidents['incident_date'])
reprisk_incidents['Year'] = reprisk_incidents['incident_date'].dt.year
reprisk_incidents['YearMonth'] = reprisk_incidents['incident_date'].dt.to_period('M')
reprisk_incidents['YearQuarter'] = reprisk_incidents['incident_date'].dt.to_period('Q')
reprisk_incidents

11718

In [8]:
reprisk_incidents.groupby(['SICS Codified Industry ', 'Year'])['story_id'].nunique()

,gvkey,isin,cusip,reprisk_id,story_id,adopter,sic,SICS Codified Industry,Codified SICS Sector,severity,reach,novelty,incident_list,incident_date,car_5,car_1,material_flag,Year,YearMonth,YearQuarter
0,210418.0,US0003752047,000375204,2,1251.0,1,3613.0,Electrical & Electronic Equipment,Resource Transformation,1.0,2.0,1.0,"['impacts_on_communities', 'impacts_on_landsca...",2007-06-19,0.023782,0.013378,0,2007.0,2007-06,2007Q2
1,210418.0,US0003752047,000375204,2,1305.0,1,3613.0,Electrical & Electronic Equipment,Resource Transformation,2.0,2.0,2.0,['human_rights_abuses'],2007-07-01,NaN,NaN,0,2007.0,2007-07,2007Q3
2,210418.0,US0003752047,000375204,2,4029.0,1,3613.0,Electrical & Electronic Equipment,Resource Transformation,2.0,1.0,2.0,"['human_rights_abuses', 'impacts_on_communitie...",2008-04-01,-0.021834,-0.025246,1,2008.0,2008-04,2008Q2
3,210418.0,US0003752047,000375204,2,4488.0,1,3613.0,Electrical & Electronic Equipment,Resource Transformation,2.0,1.0,2.0,"['controversial_prod_services', 'money_related...",2008-05-01,0.011532,-0.011851,1,2008.0,2008-05,2008Q2
4,210418.0,US0003752047,000375204,2,4756.0,1,3613.0,Electrical & Electronic Equipment,Resource Transformation,1.0,3.0,2.0,"['money_related', 'violation_of_natl_legislati...",2007-01-20,NaN,NaN,1,2007.0,2007-01,2007Q1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331365,355240.0,THB131010001,NaN,2683218,NaN,1,5900.0,Coal Operations,Extractives & Minerals Processing,0.0,0.0,0.0,[],NaT,NaN,NaN,0,NaN,NaT,NaT
331366,349631.0,PK0126301016,NaN,2684126,NaN,1,3411.0,Containers & Packaging,Resource Transformation,0.0,0.0,0.0,[],NaT,NaN,NaN,0,NaN,NaT,NaT
331367,350690.0,GB00BLNNFY18,NaN,2685085,NaN,1,6726.0,Household & Personal Products,Consumer Goods,0.0,0.0,0.0,[],NaT,NaN,NaN,0,NaN,NaT,NaT
331368,350726.0,AU0000180200,NaN,2685460,NaN,1,1400.0,Chemicals,Resource Transformation,0.0,0.0,0.0,[],NaT,NaN,NaN,0,NaN,NaT,NaT


# 1. Calculate rolling window

In [10]:
reprisk_incidents['incident_list'] = reprisk_incidents['incident_list'].apply(ast.literal_eval)
exploded = reprisk_incidents[['SICS Codified Industry ', 'Year', 'YearQuarter', 'YearMonth', 'reach', 'car_1','car_5','severity','incident_list']] \
           .explode('incident_list') \
           .rename(columns={'incident_list':'type'})

In [11]:
exploded

In [12]:
def pick_types_by_rolling_12m_period_dropna_balanced(
    df,
    industry_col='SICS Codified Industry ',  # note the trailing space (matches your df)
    ym_col='YearMonth',                      # must be Period[M]
    type_col='type',
    metrics=('reach','severity','car_1','car_5'),
    window=12
):
    d = df.copy()
    d[ym_col] = d[ym_col].astype('period[M]')
    d = d[[industry_col, ym_col, type_col, *metrics]].copy()

    # numeric coercion, keep NaNs
    for m in metrics:
        d[m] = pd.to_numeric(d[m], errors='coerce')

    # aggregate to unique (industry, type, month)
    d = (
        d.groupby([industry_col, type_col, ym_col], as_index=False)[list(metrics)]
         .sum(min_count=1)
    )

    # ---- Balanced panel over global YearMonth (for existing (industry,type) pairs only)
    global_idx = pd.period_range(d[ym_col].min(), d[ym_col].max(), freq='M')
    pairs = d[[industry_col, type_col]].drop_duplicates()
    full = (
        pairs.assign(_k=1)
             .merge(pd.DataFrame({ym_col: global_idx, '_k':1}), on='_k')
             .drop(columns='_k')
             .set_index([industry_col, type_col, ym_col])
    )
    base = d.set_index([industry_col, type_col, ym_col])
    d = full.join(base, how='left').reset_index()

    # rolling 12m sums by (industry, type); NaNs remain NaN
    d = d.sort_values([industry_col, type_col, ym_col])
    for m in metrics:
        d[f'roll12_{m}'] = (
            d.groupby([industry_col, type_col], group_keys=False)[m]
             .transform(lambda s: s.rolling(window, min_periods=1).sum())
        )

    by = [industry_col, ym_col]

    def pick_one(rollcol, take='max'):
        tmp = d[[industry_col, ym_col, type_col, rollcol]].dropna(subset=[rollcol]).copy()
        # tie-breaker: alphabetical type; for min we sort ascending by value, for max we invert via sort order
        asc_val = (take == 'min')
        tmp = tmp.sort_values(by + [rollcol, type_col],
                              ascending=[True, True, asc_val, True])
        picked = tmp.drop_duplicates(subset=by, keep='first')
        metric = rollcol.replace('roll12_', '')
        picked = picked.rename(columns={
            type_col: f'{metric}_type',
            rollcol:  f'{metric}_roll12'
        })
        return picked[[industry_col, ym_col, f'{metric}_type', f'{metric}_roll12']]

    reach_out    = pick_one('roll12_reach',    'max')
    severity_out = pick_one('roll12_severity', 'max')
    car1_out     = pick_one('roll12_car_1',    'min')
    car5_out     = pick_one('roll12_car_5',    'min')

    res = (reach_out.merge(severity_out, on=by, how='outer')
                    .merge(car1_out,     on=by, how='outer')
                    .merge(car5_out,     on=by, how='outer')
                    .sort_values(by)
                    .reset_index(drop=True))
    return res


,SICS Codified Industry,YearMonth,reach_type,reach_roll12,severity_type,severity_roll12,car_1_type,car_1_roll12,car_5_type,car_5_roll12
0,Advertising & Marketing,2007-12,discrimination_in_employment,1.0,discrimination_in_employment,1.0,discrimination_in_employment,-0.006904,discrimination_in_employment,-0.000448
1,Advertising & Marketing,2008-01,discrimination_in_employment,1.0,discrimination_in_employment,1.0,discrimination_in_employment,-0.006904,discrimination_in_employment,-0.000448
2,Advertising & Marketing,2008-02,discrimination_in_employment,1.0,discrimination_in_employment,1.0,discrimination_in_employment,-0.006904,discrimination_in_employment,-0.000448
3,Advertising & Marketing,2008-03,discrimination_in_employment,1.0,discrimination_in_employment,1.0,discrimination_in_employment,-0.006904,discrimination_in_employment,-0.000448
4,Advertising & Marketing,2008-04,discrimination_in_employment,1.0,discrimination_in_employment,1.0,discrimination_in_employment,-0.006904,discrimination_in_employment,-0.000448
...,...,...,...,...,...,...,...,...,...,...
13044,Water Utilities & Services,2023-08,local_pollution,96.0,impacts_on_communities,67.0,impacts_on_communities,-0.047760,impacts_on_communities,-0.110693
13045,Water Utilities & Services,2023-09,local_pollution,98.0,impacts_on_communities,66.0,discrimination_in_employment,-0.011628,occupational_health,-0.109424
13046,Water Utilities & Services,2023-10,violation_of_natl_legislation,114.0,violation_of_natl_legislation,77.0,discrimination_in_employment,-0.011628,occupational_health,-0.109424
13047,Water Utilities & Services,2023-11,violation_of_natl_legislation,117.0,violation_of_natl_legislation,78.0,discrimination_in_employment,-0.011628,occupational_health,-0.109424


In [42]:
# ---------------- Example usage ----------------
result = pick_types_by_rolling_12m_period_dropna_balanced(exploded)
result

,SICS Codified Industry,YearMonth,reach_type,reach_roll12,severity_type,severity_roll12,car_1_type,car_1_roll12,car_5_type,car_5_roll12
0,Advertising & Marketing,2007-12,discrimination_in_employment,1.0,discrimination_in_employment,1.0,discrimination_in_employment,-0.006904,discrimination_in_employment,-0.000448
1,Advertising & Marketing,2008-01,discrimination_in_employment,1.0,discrimination_in_employment,1.0,discrimination_in_employment,-0.006904,discrimination_in_employment,-0.000448
2,Advertising & Marketing,2008-02,discrimination_in_employment,1.0,discrimination_in_employment,1.0,discrimination_in_employment,-0.006904,discrimination_in_employment,-0.000448
3,Advertising & Marketing,2008-03,discrimination_in_employment,1.0,discrimination_in_employment,1.0,discrimination_in_employment,-0.006904,discrimination_in_employment,-0.000448
4,Advertising & Marketing,2008-04,discrimination_in_employment,1.0,discrimination_in_employment,1.0,discrimination_in_employment,-0.006904,discrimination_in_employment,-0.000448
...,...,...,...,...,...,...,...,...,...,...
13044,Water Utilities & Services,2023-08,local_pollution,96.0,impacts_on_communities,67.0,impacts_on_communities,-0.047760,impacts_on_communities,-0.110693
13045,Water Utilities & Services,2023-09,local_pollution,98.0,impacts_on_communities,66.0,discrimination_in_employment,-0.011628,occupational_health,-0.109424
13046,Water Utilities & Services,2023-10,violation_of_natl_legislation,114.0,violation_of_natl_legislation,77.0,discrimination_in_employment,-0.011628,occupational_health,-0.109424
13047,Water Utilities & Services,2023-11,violation_of_natl_legislation,117.0,violation_of_natl_legislation,78.0,discrimination_in_employment,-0.011628,occupational_health,-0.109424


In [13]:
result.groupby('SICS Codified Industry ')['car_5_type'].nunique()

,gvkey,isin,cusip,reprisk_id,story_id,adopter,sic,SICS Codified Industry,Codified SICS Sector,severity,...,YearMonth,YearQuarter,reach_type,reach_roll12,severity_type,severity_roll12,car_1_type,car_1_roll12,car_5_type,car_5_roll12
0,210418.0,US0003752047,000375204,2,1251.0,1,3613.0,Electrical & Electronic Equipment,Resource Transformation,1.0,...,2007-06,2007Q2,violation_of_natl_legislation,28.0,violation_of_natl_legislation,18.0,impacts_on_communities,0.009035,impacts_on_communities,0.016264
1,210418.0,US0003752047,000375204,2,1305.0,1,3613.0,Electrical & Electronic Equipment,Resource Transformation,2.0,...,2007-07,2007Q3,violation_of_natl_legislation,28.0,violation_of_natl_legislation,18.0,impacts_on_communities,0.009035,impacts_on_communities,0.016264
2,210418.0,US0003752047,000375204,2,4029.0,1,3613.0,Electrical & Electronic Equipment,Resource Transformation,2.0,...,2008-04,2008Q2,impacts_on_landscapes,43.0,human_rights_abuses,41.0,climate_ghg_pollution,-0.129756,impacts_on_communities,-0.282756
3,210418.0,US0003752047,000375204,2,4488.0,1,3613.0,Electrical & Electronic Equipment,Resource Transformation,2.0,...,2008-05,2008Q2,human_rights_abuses,47.0,human_rights_abuses,54.0,climate_ghg_pollution,-0.129756,supply_chain_issues,-0.250865
4,210418.0,US0003752047,000375204,2,4756.0,1,3613.0,Electrical & Electronic Equipment,Resource Transformation,1.0,...,2007-01,2007Q1,violation_of_natl_legislation,21.0,violation_of_natl_legislation,11.0,impacts_on_communities,-0.004342,impacts_on_communities,-0.007518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331365,355240.0,THB131010001,NaN,2683218,NaN,1,5900.0,Coal Operations,Extractives & Minerals Processing,0.0,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
331366,349631.0,PK0126301016,NaN,2684126,NaN,1,3411.0,Containers & Packaging,Resource Transformation,0.0,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
331367,350690.0,GB00BLNNFY18,NaN,2685085,NaN,1,6726.0,Household & Personal Products,Consumer Goods,0.0,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
331368,350726.0,AU0000180200,NaN,2685460,NaN,1,1400.0,Chemicals,Resource Transformation,0.0,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Merge back to original dataframe
final = reprisk_incidents.merge(
    result,
    on=['SICS Codified Industry ', 'YearMonth'],
    how='left'
)
final

In [15]:
def _in_list(t, lst):
    if pd.isna(t):
        return 0
    return int(str(t) in lst)

# --- 3) create materiality flags
final['materiality_reach']    = [_in_list(t, L) for t, L in zip(final['reach_type'],    final['incident_list'])]
final['materiality_severity'] = [_in_list(t, L) for t, L in zip(final['severity_type'], final['incident_list'])]
final['materiality_car_1']    = [_in_list(t, L) for t, L in zip(final['car_1_type'],    final['incident_list'])]
final['materiality_car_5']    = [_in_list(t, L) for t, L in zip(final['car_5_type'],    final['incident_list'])]
final

In [60]:
final.to_csv('data/incidents_rolling_firm.csv', index=False)

# 2. Return on firm level

In [30]:
final = pd.read_csv('data/incidents_rolling_firm.csv')

Loading library list...
Done


In [172]:
import wrds
from pathlib import Path
from dotenv import load_dotenv
import os
from itertools import islice

project_root = Path(r"E:\GermanBusinessPanelTeam\Schrader\Forschung\ESGmateriality")

# ── Load WRDS creds from wrds.env in the project root ─────────────────────────
env_path = project_root / "wrds.env"
load_dotenv(dotenv_path=env_path)
# ── Connect to WRDS ───────────────────────────────────────────────────────────
db = wrds.Connection(
    wrds_username=os.getenv("WRDS_YALE_USERNAME"),
    wrds_password=os.getenv("WRDS_YALE_PASSWORD")
)

Loading library list...
Done


In [31]:
db.list_libraries()

In [32]:
# --- 1) Unique CUSIPs and date bounds (work on copies; your df_incidents stays untouched)
cusips = pd.Series(final['cusip'].dropna().unique(), dtype=object).tolist()

dates = pd.to_datetime(pd.Series(final['incident_date'].dropna()), errors='coerce').dropna()
start_date = dates.min().strftime('%Y-%m-%d')
end_date   = (dates.max() + pd.Timedelta(days=1)).strftime('%Y-%m-%d')  # include +1 day

# --- 2) Query CRSP.dsf in manageable chunks
def chunks(seq, size):
    it = iter(seq)
    while True:
        block = list(islice(it, size))
        if not block:
            break
        yield block

dfs = []
for block in chunks(cusips, 2000):  # adjust chunk size if needed
    cusip_list_sql = "(" + ",".join(f"'{c}'" for c in block) + ")"
    sql = f"""
        SELECT cusip, date, ret
        FROM crsp.dsf
        WHERE date BETWEEN '{start_date}' AND '{end_date}'
          AND cusip IN {cusip_list_sql}
    """
    dfs.append(db.raw_sql(sql, date_cols=['date']))

# --- 3) Concatenate & tidy
all_returns = (
    pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame(columns=['cusip','date','ret'])
    .astype({'cusip':'object'})
)
all_returns['ret'] = pd.to_numeric(all_returns['ret'], errors='coerce')
all_returns = all_returns.sort_values(['cusip','date']).reset_index(drop=True)

C:\Users\PascalSchrader.WIN-T1MVMVKE25U.000\AppData\Local\Temp\9\ipykernel_46028\80811019.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(fill_and_bfill)


In [33]:
all_returns

In [34]:
def fill_and_bfill(g):
    # full daily calendar from first to last observed date
    full_idx = pd.date_range(g['date'].min(), g['date'].max(), freq='D')
    g2 = (g.set_index('date')
            .reindex(full_idx)           # adds missing days as NaNs
            .rename_axis('date'))
    # keep cusip, then backfill ret (use next trading day's value)
    g2['cusip'] = g2['cusip'].ffill().bfill()
    g2['ret']   = g2['ret'].bfill()
    return g2.reset_index()

df_filled = (all_returns.groupby('cusip', group_keys=False)
               .apply(fill_and_bfill)
               .sort_values(['cusip','date'])
               .reset_index(drop=True))

(9840767, 3)


,cusip,date,ret
0,00030710,2014-10-02,<NA>
1,00030710,2014-10-03,0.008108
2,00030710,2014-10-06,0.031635
3,00030710,2014-10-07,0.0
4,00030710,2014-10-08,-0.005717


In [35]:
df_filled

,cusip,date,ar,next_date,next_ar
0,00030710,2014-10-02,<NA>,2014-10-03,-0.000988
1,00030710,2014-10-03,-0.000988,2014-10-06,0.033217
2,00030710,2014-10-06,0.033217,2014-10-07,0.01499
3,00030710,2014-10-07,0.01499,2014-10-08,-0.021572
4,00030710,2014-10-08,-0.021572,2014-10-09,0.027493


In [36]:
# final must have: 'cusip', 'incident_date'
cusips = pd.Series(final['cusip'].dropna().unique()).astype(str)

d = pd.to_datetime(final['incident_date'], errors='coerce').dropna()
dates = pd.Index(d.dt.normalize().unique()).sort_values()  # unique YYYY-MM-DD (midnight)

pairs = pd.MultiIndex.from_product([cusips, dates], names=['cusip', 'incident_date']).to_frame(index=False)

print(len(cusips), "unique CUSIPs")
print(len(dates), "unique incident dates")
print(len(pairs), "Cartesian pairs")
pairs.head()


,cusip,date,ret,vwretd,ar,next_ar,next_date,CAR_0_1
0,00030710,2014-10-02,<NA>,0.000775,<NA>,-0.000988,2014-10-03,<NA>
1,00030710,2014-10-03,0.008108,0.009096,-0.000988,0.033217,2014-10-06,0.032229
2,00030710,2014-10-06,0.031635,-0.001582,0.033217,0.01499,2014-10-07,0.048207
3,00030710,2014-10-07,0.0,-0.01499,0.01499,-0.021572,2014-10-08,-0.006582
4,00030710,2014-10-08,-0.005717,0.015855,-0.021572,0.027493,2014-10-09,0.005921
...,...,...,...,...,...,...,...,...
9840762,V7780T10,2023-12-22,0.008575,0.002434,0.006141,0.000783,2023-12-26,0.006924
9840763,V7780T10,2023-12-26,0.006006,0.005223,0.000783,0.005299,2023-12-27,0.006082
9840764,V7780T10,2023-12-27,0.007288,0.001989,0.005299,0.000496,2023-12-28,0.005795
9840765,V7780T10,2023-12-28,0.000385,-0.000111,0.000496,0.000356,2023-12-29,0.000852


In [101]:
# Date window: [min(incident_date), max(incident_date)+1 day] (add a small buffer if you like)
start_date = dates.min().strftime('%Y-%m-%d')
end_date   = (dates.max() + pd.Timedelta(days=1)).strftime('%Y-%m-%d')

# Single query version (simple). If your CUSIP list is huge and SQL IN is too long, split manually.
cusip_list_sql = "(" + ",".join(f"'{c}'" for c in cusips.tolist()) + ")"

sql_dsf = f"""
    SELECT cusip, date, ret
    FROM crsp.dsf
    WHERE date BETWEEN '{start_date}' AND '{end_date}'
      AND cusip IN {cusip_list_sql}
"""
dsf = db.raw_sql(sql_dsf, date_cols=['date'])

# Tidy
dsf['cusip'] = dsf['cusip'].astype(str)
dsf['ret']   = pd.to_numeric(dsf['ret'], errors='coerce')
dsf = dsf.dropna(subset=['cusip','date']).sort_values(['cusip','date']).reset_index(drop=True)

print(dsf.shape)
dsf.head()


(9840767, 3)


,cusip,date,ret
0,00030710,2014-10-02,<NA>
1,00030710,2014-10-03,0.008108
2,00030710,2014-10-06,0.031635
3,00030710,2014-10-07,0.0
4,00030710,2014-10-08,-0.005717


In [102]:
sql_mkt = f"""
    SELECT date, vwretd
    FROM crsp.dsi
    WHERE date BETWEEN '{start_date}' AND '{end_date}'
"""
mkt = db.raw_sql(sql_mkt, date_cols=['date'])
mkt['vwretd'] = pd.to_numeric(mkt['vwretd'], errors='coerce')

dsf_ar = dsf.merge(mkt, on='date', how='left')
dsf_ar['ar'] = dsf_ar['ret'] - dsf_ar['vwretd']

# For Day 1 we’ll need the next trading day’s AR; compute it via groupby().shift(-1)
dsf_ar = dsf_ar.sort_values(['cusip','date']).reset_index(drop=True)
dsf_ar['next_ar']   = dsf_ar.groupby('cusip')['ar'].shift(-1)
dsf_ar['next_date'] = dsf_ar.groupby('cusip')['date'].shift(-1)

dsf_ar[['cusip','date','ar','next_date','next_ar']].head()


,cusip,date,ar,next_date,next_ar
0,00030710,2014-10-02,<NA>,2014-10-03,-0.000988
1,00030710,2014-10-03,-0.000988,2014-10-06,0.033217
2,00030710,2014-10-06,0.033217,2014-10-07,0.01499
3,00030710,2014-10-07,0.01499,2014-10-08,-0.021572
4,00030710,2014-10-08,-0.021572,2014-10-09,0.027493


In [109]:
# df has: ['cusip','date','ar','next_date','next_ar']
df = dsf_ar.copy()
df['CAR_0_1'] = df['ar'] + df['next_ar']   # stays NaN if either side is NaN
df

,cusip,date,ret,vwretd,ar,next_ar,next_date,CAR_0_1
0,00030710,2014-10-02,<NA>,0.000775,<NA>,-0.000988,2014-10-03,<NA>
1,00030710,2014-10-03,0.008108,0.009096,-0.000988,0.033217,2014-10-06,0.032229
2,00030710,2014-10-06,0.031635,-0.001582,0.033217,0.01499,2014-10-07,0.048207
3,00030710,2014-10-07,0.0,-0.01499,0.01499,-0.021572,2014-10-08,-0.006582
4,00030710,2014-10-08,-0.005717,0.015855,-0.021572,0.027493,2014-10-09,0.005921
...,...,...,...,...,...,...,...,...
9840762,V7780T10,2023-12-22,0.008575,0.002434,0.006141,0.000783,2023-12-26,0.006924
9840763,V7780T10,2023-12-26,0.006006,0.005223,0.000783,0.005299,2023-12-27,0.006082
9840764,V7780T10,2023-12-27,0.007288,0.001989,0.005299,0.000496,2023-12-28,0.005795
9840765,V7780T10,2023-12-28,0.000385,-0.000111,0.000496,0.000356,2023-12-29,0.000852


In [111]:
pairs_merged = pairs.merge(df[['cusip', 'date', 'CAR_0_1']],
                           left_on=['cusip','incident_date'],
                           right_on=['cusip','date'],
                           how='inner')
pairs_merged

,cusip,incident_date,date,CAR_0_1
0,00030710,2014-10-02,2014-10-02,<NA>
1,00030710,2014-10-03,2014-10-03,0.032229
2,00030710,2014-10-06,2014-10-06,0.048207
3,00030710,2014-10-07,2014-10-07,-0.006582
4,00030710,2014-10-08,2014-10-08,0.005921
...,...,...,...,...
9822610,V7780T10,2023-12-22,2023-12-22,0.006924
9822611,V7780T10,2023-12-26,2023-12-26,0.006082
9822612,V7780T10,2023-12-27,2023-12-27,0.005795
9822613,V7780T10,2023-12-28,2023-12-28,0.000852


In [159]:
final.drop_duplicates(subset=['cusip', 'incident_date'], inplace=True)
pairs_incidents = pairs_merged.merge(final[['cusip', 'incident_date', 'severity', 'reach', 'novelty', 'material_flag', 'materiality_car_5', 'materiality_reach', 'materiality_car_1', 'materiality_severity']], on=['cusip', 'incident_date'], how='left')
pairs_incidents

,cusip,incident_date,date,CAR_0_1,severity,reach,novelty,material_flag,materiality_car_5,materiality_reach,materiality_car_1,materiality_severity
0,00030710,2014-10-02,2014-10-02,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00030710,2014-10-03,2014-10-03,0.032229,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00030710,2014-10-06,2014-10-06,0.048207,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00030710,2014-10-07,2014-10-07,-0.006582,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00030710,2014-10-08,2014-10-08,0.005921,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
9822610,V7780T10,2023-12-22,2023-12-22,0.006924,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9822611,V7780T10,2023-12-26,2023-12-26,0.006082,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9822612,V7780T10,2023-12-27,2023-12-27,0.005795,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9822613,V7780T10,2023-12-28,2023-12-28,0.000852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [160]:
# Indicator column if material_flag is not na
pairs_incidents['incident'] = np.where(pairs_incidents['material_flag'].notna(), 1, 0)

In [164]:
# Merge gvkey
pairs_industry = pairs_incidents.merge(reprisk_incidents[['cusip', 'gvkey']].drop_duplicates(), on='cusip', how='left')

In [165]:
# Merge Industry and Sector
pairs_industry = pairs_industry.merge(reprisk_incidents[['cusip', 'SICS Codified Industry ', 'Codified SICS Sector ']].drop_duplicates(), on='cusip', how='left')
pairs_industry.notna().sum()

cusip                      9822615
incident_date              9822615
date                       9822615
CAR_0_1                    9747738
severity                     98505
reach                        98505
novelty                      98505
material_flag                98505
materiality_car_5            98505
materiality_reach            98505
materiality_car_1            98505
materiality_severity         98505
incident                   9822615
gvkey                      9822615
SICS Codified Industry     9768786
Codified SICS Sector       9768786
dtype: int64

In [166]:
pairs_industry.dropna(subset=['SICS Codified Industry ', 'Codified SICS Sector ', 'CAR_0_1'], inplace=True)

In [167]:
# Create indicator column if material incident in same SICS Industry

df = pairs_industry.copy()

mat_cols = ['material_flag', 'materiality_car_5', 'materiality_reach',
            'materiality_car_1', 'materiality_severity', 'incident']

# 1) Clean to 0/1 at firm-day (NaN -> 0; any nonzero -> 1)
tmp = df[['SICS Codified Industry ', 'date'] + mat_cols].copy()
for c in mat_cols:
    tmp[c] = (pd.to_numeric(tmp[c], errors='coerce').fillna(0) != 0).astype(int)

# 2) Industry-date flags: any firm in industry had an incident today under each definition
ind_flags = (
    tmp.groupby(['SICS Codified Industry ', 'date'])[mat_cols]
       .max()
       .reset_index()
       .rename(columns={c: f'industry_{c}' for c in mat_cols})
)

# 3) Merge back to firm-day
df = df.merge(ind_flags, on=['SICS Codified Industry ', 'date'], how='left')

df

,cusip,incident_date,date,CAR_0_1,severity,reach,novelty,material_flag,materiality_car_5,materiality_reach,...,incident,gvkey,SICS Codified Industry,Codified SICS Sector,industry_material_flag,industry_materiality_car_5,industry_materiality_reach,industry_materiality_car_1,industry_materiality_severity,industry_incident
0,00030710,2014-10-03,2014-10-03,0.032229,NaN,NaN,NaN,NaN,NaN,NaN,...,0,178698.0,Health Care Delivery,Health Care,0,0,0,0,0,0
1,00030710,2014-10-06,2014-10-06,0.048207,NaN,NaN,NaN,NaN,NaN,NaN,...,0,178698.0,Health Care Delivery,Health Care,0,0,0,0,0,0
2,00030710,2014-10-07,2014-10-07,-0.006582,NaN,NaN,NaN,NaN,NaN,NaN,...,0,178698.0,Health Care Delivery,Health Care,0,0,0,0,0,0
3,00030710,2014-10-08,2014-10-08,0.005921,NaN,NaN,NaN,NaN,NaN,NaN,...,0,178698.0,Health Care Delivery,Health Care,0,0,0,0,0,0
4,00030710,2014-10-09,2014-10-09,0.027839,NaN,NaN,NaN,NaN,NaN,NaN,...,0,178698.0,Health Care Delivery,Health Care,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9693933,V7780T10,2023-12-21,2023-12-21,0.047159,NaN,NaN,NaN,NaN,NaN,NaN,...,0,28191.0,Marine Transportation,Transportation,0,0,0,0,0,0
9693934,V7780T10,2023-12-22,2023-12-22,0.006924,NaN,NaN,NaN,NaN,NaN,NaN,...,0,28191.0,Marine Transportation,Transportation,0,0,0,0,0,0
9693935,V7780T10,2023-12-26,2023-12-26,0.006082,NaN,NaN,NaN,NaN,NaN,NaN,...,0,28191.0,Marine Transportation,Transportation,0,0,0,0,0,0
9693936,V7780T10,2023-12-27,2023-12-27,0.005795,NaN,NaN,NaN,NaN,NaN,NaN,...,0,28191.0,Marine Transportation,Transportation,0,0,0,0,0,0


In [168]:
# Fill material columns NaNs with 0 (no incident -> no material incident)
# select only columns that already start with 'materiality_'
mat_cols = [c for c in df.columns if c.startswith('materiality_')]
if 'material_flag' in df.columns:
    mat_cols.append('material_flag')

# coerce to numeric, fill NAs with 0, cast to int
df[mat_cols] = (
    df[mat_cols]
      .apply(lambda s: pd.to_numeric(s, errors='coerce'))
      .fillna(0)
      .astype(int)
)
df

,cusip,incident_date,date,CAR_0_1,severity,reach,novelty,material_flag,materiality_car_5,materiality_reach,...,incident,gvkey,SICS Codified Industry,Codified SICS Sector,industry_material_flag,industry_materiality_car_5,industry_materiality_reach,industry_materiality_car_1,industry_materiality_severity,industry_incident
0,00030710,2014-10-03,2014-10-03,0.032229,NaN,NaN,NaN,0,0,0,...,0,178698.0,Health Care Delivery,Health Care,0,0,0,0,0,0
1,00030710,2014-10-06,2014-10-06,0.048207,NaN,NaN,NaN,0,0,0,...,0,178698.0,Health Care Delivery,Health Care,0,0,0,0,0,0
2,00030710,2014-10-07,2014-10-07,-0.006582,NaN,NaN,NaN,0,0,0,...,0,178698.0,Health Care Delivery,Health Care,0,0,0,0,0,0
3,00030710,2014-10-08,2014-10-08,0.005921,NaN,NaN,NaN,0,0,0,...,0,178698.0,Health Care Delivery,Health Care,0,0,0,0,0,0
4,00030710,2014-10-09,2014-10-09,0.027839,NaN,NaN,NaN,0,0,0,...,0,178698.0,Health Care Delivery,Health Care,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9693933,V7780T10,2023-12-21,2023-12-21,0.047159,NaN,NaN,NaN,0,0,0,...,0,28191.0,Marine Transportation,Transportation,0,0,0,0,0,0
9693934,V7780T10,2023-12-22,2023-12-22,0.006924,NaN,NaN,NaN,0,0,0,...,0,28191.0,Marine Transportation,Transportation,0,0,0,0,0,0
9693935,V7780T10,2023-12-26,2023-12-26,0.006082,NaN,NaN,NaN,0,0,0,...,0,28191.0,Marine Transportation,Transportation,0,0,0,0,0,0
9693936,V7780T10,2023-12-27,2023-12-27,0.005795,NaN,NaN,NaN,0,0,0,...,0,28191.0,Marine Transportation,Transportation,0,0,0,0,0,0


In [137]:
db.describe_table(library='ff', table='factors_daily')

Approximately 26066 rows in ff.factors_daily.


,name,nullable,type,comment
0,date,True,DATE,Date
1,mktrf,True,"NUMERIC(8, 6)",Excess Return on the Market
2,smb,True,"NUMERIC(8, 6)",Small-Minus-Big Return
3,hml,True,"NUMERIC(8, 6)",High-Minus-Low Return
4,rf,True,"NUMERIC(7, 5)",Risk-Free Return Rate (One Month Treasury Bill...
5,umd,True,"NUMERIC(8, 6)",Momentum


In [169]:
def fetch_ff3_mom_daily(db, start_date, end_date):
    """Return daily FF3+RF+UMD (all in decimals) for [start_date, end_date]."""
    sql_ff = f"""
        SELECT date, mktrf, smb, hml, rf, umd
        FROM ff.factors_daily
        WHERE date BETWEEN '{start_date}' AND '{end_date}'
    """
    ff = db.raw_sql(sql_ff, date_cols=['date']).sort_values('date')


    # Convert % to decimals
    for c in ['mktrf', 'smb', 'hml', 'rf', 'umd']:
        ff[c] = pd.to_numeric(ff[c], errors='coerce') / 100.0

    return ff

In [173]:
# ---- Example usage with your event list df_events ----
evt = df.copy()             # your table above
evt['date'] = pd.to_datetime(evt['date'])

start_date = evt['date'].min()
end_date   = evt['date'].max()

factors = fetch_ff3_mom_daily(db, start_date, end_date)
factors

,date,mktrf,smb,hml,rf,umd
0,2007-01-03,-0.000005,0.000003,0.00002,0.000002,-0.000045
1,2007-01-04,0.000016,0.000023,-0.000051,0.000002,-0.000056
2,2007-01-05,-0.000073,-0.000094,-0.000029,0.000002,0.000005
3,2007-01-08,0.000024,-0.000009,0.000004,0.000002,0.000032
4,2007-01-09,0.0,0.000029,-0.000031,0.000002,0.000025
...,...,...,...,...,...,...
4272,2023-12-21,0.000109,0.000074,-0.000004,0.000002,-0.00002
4273,2023-12-22,0.00002,0.000064,0.00001,0.000002,-0.000048
4274,2023-12-26,0.000048,0.00007,0.000044,0.000002,-0.000021
4275,2023-12-27,0.000016,0.000013,0.000011,0.000002,0.000011


In [174]:
# 2) Build next-day factors and 2-day sums (t + t+1 trading day)
sum_cols = ['mktrf','smb','hml','rf', 'umd']

for c in sum_cols:
    factors[f'next_{c}'] = factors[c].shift(-1)

ff_2d = factors.assign(
    MKT_2d = factors['mktrf'] + factors['next_mktrf'],
    SMB_2d = factors['smb']   + factors['next_smb'],
    HML_2d = factors['hml']   + factors['next_hml'],
    RF_2d  = factors['rf']    + factors['next_rf'],
    UMD_2d = factors['umd']  + factors['next_umd']
).dropna(subset=['next_mktrf'])  # drop last day (no t+1)

keep_cols = ['date','MKT_2d','SMB_2d','HML_2d','RF_2d', 'UMD_2d']
ff_2d = ff_2d[keep_cols]
ff_2d

,date,MKT_2d,SMB_2d,HML_2d,RF_2d,UMD_2d
0,2007-01-03,0.000011,0.000026,-0.000031,0.000004,-0.000101
1,2007-01-04,-0.000057,-0.000071,-0.00008,0.000004,-0.000051
2,2007-01-05,-0.000049,-0.000103,-0.000025,0.000004,0.000037
3,2007-01-08,0.000024,0.00002,-0.000027,0.000004,0.000057
4,2007-01-09,0.000023,0.000019,-0.00005,0.000004,0.000028
...,...,...,...,...,...,...
4271,2023-12-20,-0.000046,0.000037,0.000009,0.000004,0.00004
4272,2023-12-21,0.000129,0.000138,0.000006,0.000004,-0.000068
4273,2023-12-22,0.000068,0.000134,0.000054,0.000004,-0.000069
4274,2023-12-26,0.000064,0.000083,0.000055,0.000004,-0.00001


In [175]:
# Merge by date
evt = evt.merge(ff_2d, on='date', how='left')
evt

,cusip,incident_date,date,CAR_0_1,severity,reach,novelty,material_flag,materiality_car_5,materiality_reach,...,industry_materiality_car_5,industry_materiality_reach,industry_materiality_car_1,industry_materiality_severity,industry_incident,MKT_2d,SMB_2d,HML_2d,RF_2d,UMD_2d
0,00030710,2014-10-03,2014-10-03,0.032229,NaN,NaN,NaN,0,0,0,...,0,0,0,0,0,0.000082,-0.000116,-0.000009,0.0,0.000016
1,00030710,2014-10-06,2014-10-06,0.048207,NaN,NaN,NaN,0,0,0,...,0,0,0,0,0,-0.000182,-0.000092,0.000051,0.0,-0.000019
2,00030710,2014-10-07,2014-10-07,-0.006582,NaN,NaN,NaN,0,0,0,...,0,0,0,0,0,0.000014,-0.000011,0.000009,0.0,0.000037
3,00030710,2014-10-08,2014-10-08,0.005921,NaN,NaN,NaN,0,0,0,...,0,0,0,0,0,-0.000047,-0.000039,-0.000046,0.0,0.000031
4,00030710,2014-10-09,2014-10-09,0.027839,NaN,NaN,NaN,0,0,0,...,0,0,0,0,0,-0.000346,-0.000059,0.000011,0.0,-0.000034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9693933,V7780T10,2023-12-21,2023-12-21,0.047159,NaN,NaN,NaN,0,0,0,...,0,0,0,0,0,0.000129,0.000138,0.000006,0.000004,-0.000068
9693934,V7780T10,2023-12-22,2023-12-22,0.006924,NaN,NaN,NaN,0,0,0,...,0,0,0,0,0,0.000068,0.000134,0.000054,0.000004,-0.000069
9693935,V7780T10,2023-12-26,2023-12-26,0.006082,NaN,NaN,NaN,0,0,0,...,0,0,0,0,0,0.000064,0.000083,0.000055,0.000004,-0.00001
9693936,V7780T10,2023-12-27,2023-12-27,0.005795,NaN,NaN,NaN,0,0,0,...,0,0,0,0,0,0.000015,-0.000023,0.000014,0.000004,-0.000037


In [41]:
evt.drop(columns=['incident_date', 'severity', 'reach', 'novelty'], inplace=True)

C:\Users\PascalSchrader.WIN-T1MVMVKE25U.000\AppData\Local\Temp\9\ipykernel_46028\2550280871.py:1: DtypeWarning: Columns (0,30) have mixed types. Specify dtype option on import or set low_memory=False.
  evt = pd.read_csv('Output/events_ff3mom_daily.csv')


In [42]:
evt.to_csv('Output/events_ff3mom_daily.csv', index=False)

,cusip,date,CAR_0_1,material_flag,materiality_car_5,materiality_reach,materiality_car_1,materiality_severity,incident,gvkey,...,ad_intensity,roa,rdate,InstOwn_Perc,incident_date_x,severity_x,reach_x,incident_date_y,severity_y,reach_y
0,00030710,2014-10-03,0.032229,0,0,0,0,0,0,178698.0,...,NaN,0.011624,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00030710,2014-10-06,0.048207,0,0,0,0,0,0,178698.0,...,NaN,0.011624,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00030710,2014-10-07,-0.006582,0,0,0,0,0,0,178698.0,...,NaN,0.011624,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00030710,2014-10-08,0.005921,0,0,0,0,0,0,178698.0,...,NaN,0.011624,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00030710,2014-10-09,0.027839,0,0,0,0,0,0,178698.0,...,NaN,0.011624,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9738065,V7780T10,2023-12-21,0.047159,0,0,0,0,0,0,28191.0,...,0.255918,-0.065298,2021-12-31,0.664304,NaN,NaN,NaN,NaN,NaN,NaN
9738066,V7780T10,2023-12-22,0.006924,0,0,0,0,0,0,28191.0,...,0.255918,-0.065298,2021-12-31,0.664304,NaN,NaN,NaN,NaN,NaN,NaN
9738067,V7780T10,2023-12-26,0.006082,0,0,0,0,0,0,28191.0,...,0.255918,-0.065298,2021-12-31,0.664304,NaN,NaN,NaN,NaN,NaN,NaN
9738068,V7780T10,2023-12-27,0.005795,0,0,0,0,0,0,28191.0,...,0.255918,-0.065298,2021-12-31,0.664304,NaN,NaN,NaN,NaN,NaN,NaN


,cusip,date,CAR_0_1,material_flag,materiality_car_5,materiality_reach,materiality_car_1,materiality_severity,incident,gvkey,...,btm,leverage,rd_intensity,ad_intensity,roa,rdate,InstOwn_Perc,incident_date,severity,reach
0,00030710,2014-10-03,0.032229,0,0,0,0,0,0,178698.0,...,408.913043,0.696052,0.0,NaN,0.011624,NaN,NaN,NaN,NaN,NaN
1,00030710,2014-10-06,0.048207,0,0,0,0,0,0,178698.0,...,408.913043,0.696052,0.0,NaN,0.011624,NaN,NaN,NaN,NaN,NaN
2,00030710,2014-10-07,-0.006582,0,0,0,0,0,0,178698.0,...,408.913043,0.696052,0.0,NaN,0.011624,NaN,NaN,NaN,NaN,NaN
3,00030710,2014-10-08,0.005921,0,0,0,0,0,0,178698.0,...,408.913043,0.696052,0.0,NaN,0.011624,NaN,NaN,NaN,NaN,NaN
4,00030710,2014-10-09,0.027839,0,0,0,0,0,0,178698.0,...,408.913043,0.696052,0.0,NaN,0.011624,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9706565,V7780T10,2023-12-21,0.047159,0,0,0,0,0,0,28191.0,...,0.259525,0.842349,NaN,0.255918,-0.065298,2021-12-31,0.664304,NaN,NaN,NaN
9706566,V7780T10,2023-12-22,0.006924,0,0,0,0,0,0,28191.0,...,0.259525,0.842349,NaN,0.255918,-0.065298,2021-12-31,0.664304,NaN,NaN,NaN
9706567,V7780T10,2023-12-26,0.006082,0,0,0,0,0,0,28191.0,...,0.259525,0.842349,NaN,0.255918,-0.065298,2021-12-31,0.664304,NaN,NaN,NaN
9706568,V7780T10,2023-12-27,0.005795,0,0,0,0,0,0,28191.0,...,0.259525,0.842349,NaN,0.255918,-0.065298,2021-12-31,0.664304,NaN,NaN,NaN


In [50]:
# Make unique lists
cusips = pd.Series(final['cusip'].dropna().unique(), dtype=object).reset_index(drop=True)
dates  = pd.to_datetime(final['incident_date'], errors='coerce') \
            .dropna().dt.strftime('%Y-%m-%d').drop_duplicates().sort_values().reset_index(drop=True)

# Split dates into 4 nearly equal chunks (ceiling division without extra imports)
n = len(dates)
chunk_size = (n + 3) // 22  # ceil(n/4)

for i in range(22):
    start = i * chunk_size
    end   = min((i + 1) * chunk_size, n)
    dchunk = dates[start:end]

    out_path = f"data/events_cusip_dates_part{i+1}.txt"
    with open(out_path, 'w', encoding='utf-8') as f:
        for d in dchunk:
            f.write('\n'.join(f"{c} {d}" for c in cusips))
            f.write('\n')

    print(f"part{i+1}: {len(cusips)} × {len(dchunk)} = {len(cusips)*len(dchunk)} lines -> {out_path}")


In [51]:
print(final['cusip'].nunique())
print(final['incident_date'].nunique())

,cusip,date,CAR_0_1,material_flag,materiality_car_5,materiality_reach,materiality_car_1,materiality_severity,incident,gvkey,...,InstOwn_Perc,incident_date,severity,reach,peer_severity3,peer_reach3,top_quintile_industry,top_decile_industry,top_quintile,top_decile
0,00030710,2014-10-03,0.032229,0,0,0,0,0,0,178698.0,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
1,00030710,2014-10-06,0.048207,0,0,0,0,0,0,178698.0,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
2,00030710,2014-10-07,-0.006582,0,0,0,0,0,0,178698.0,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
3,00030710,2014-10-08,0.005921,0,0,0,0,0,0,178698.0,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
4,00030710,2014-10-09,0.027839,0,0,0,0,0,0,178698.0,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9738065,V7780T10,2023-12-21,0.047159,0,0,0,0,0,0,28191.0,...,0.664304,NaN,NaN,NaN,0,0,0,0,0,0
9738066,V7780T10,2023-12-22,0.006924,0,0,0,0,0,0,28191.0,...,0.664304,NaN,NaN,NaN,0,0,0,0,0,0
9738067,V7780T10,2023-12-26,0.006082,0,0,0,0,0,0,28191.0,...,0.664304,NaN,NaN,NaN,0,0,0,0,0,0
9738068,V7780T10,2023-12-27,0.005795,0,0,0,0,0,0,28191.0,...,0.664304,NaN,NaN,NaN,0,0,0,0,0,0


# 3. Aggregate on industry level

In [17]:
df = final.copy()


# Coerce flags to numeric and treat missing as 0
flag_cols = [
    'material_flag',
    'materiality_car_1', 'materiality_car_5', 'materiality_reach', 'materiality_severity'
]
df[flag_cols] = df[flag_cols].apply(pd.to_numeric, errors='coerce').fillna(0)

key = ["gvkey", "YearMonth"]

# 1) Aggregate incidents per firm-quarter
agg_fq = (
    df.groupby(key, dropna=False)
      .agg(
          n_material          = ('material_flag',         lambda s: (s == 1).sum()),
          n_nonmaterial       = ('material_flag',         lambda s: (s == 0).sum()),
          n_car_1_material    = ('materiality_car_1',     lambda s: (s == 1).sum()),
          n_car_5_material    = ('materiality_car_5',     lambda s: (s == 1).sum()),
          n_reach_material    = ('materiality_reach',     lambda s: (s == 1).sum()),
          n_severity_material = ('materiality_severity',  lambda s: (s == 1).sum()),
      )
      .reset_index()
)

# 2) Build the BALANCED universe = all firms × all quarters (global min..max)
firms = (
    df['gvkey']
    .dropna()
    .unique()
)
qmin = df['YearMonth'].min()
qmax = df['YearMonth'].max()
quarters = pd.period_range(qmin, qmax, freq='M')

balanced_idx = pd.MultiIndex.from_product([firms, quarters], names=key)
balanced_universe = pd.DataFrame(index=balanced_idx).reset_index()

# 3) Merge counts onto balanced universe and fill missing with zeros
reprisk = (
    balanced_universe
      .merge(agg_fq, on=key, how='left')
      .fillna(0)
      .sort_values(key)
      .reset_index(drop=True)
)

# 4) Cast counts to ints
count_cols = [
    'n_material','n_nonmaterial','n_car_1_material','n_car_5_material','n_reach_material','n_severity_material'
]
reprisk[count_cols] = reprisk[count_cols].astype('int16')

reprisk

,gvkey,YearMonth,n_material,n_nonmaterial,n_car_1_material,n_car_5_material,n_reach_material,n_severity_material,cusip,SICS Codified Industry,Codified SICS Sector
0,1004.0,2007-01,0,0,0,0,0,0,000361105,Industrial Machinery & Goods,Resource Transformation
1,1004.0,2007-02,0,0,0,0,0,0,000361105,Industrial Machinery & Goods,Resource Transformation
2,1004.0,2007-03,0,0,0,0,0,0,000361105,Industrial Machinery & Goods,Resource Transformation
3,1004.0,2007-04,0,0,0,0,0,0,000361105,Industrial Machinery & Goods,Resource Transformation
4,1004.0,2007-05,0,0,0,0,0,0,000361105,Industrial Machinery & Goods,Resource Transformation
...,...,...,...,...,...,...,...,...,...,...,...
10643287,367496.0,2023-08,0,0,0,0,0,0,NaN,Real Estate,Infrastructure
10643288,367496.0,2023-09,0,0,0,0,0,0,NaN,Real Estate,Infrastructure
10643289,367496.0,2023-10,0,0,0,0,0,0,NaN,Real Estate,Infrastructure
10643290,367496.0,2023-11,0,0,0,0,0,0,NaN,Real Estate,Infrastructure


In [18]:
reprisk_merged = reprisk.merge(reprisk_incidents[['gvkey', 'cusip', 'SICS Codified Industry ', 'Codified SICS Sector ']].drop_duplicates(), on='gvkey', how='left')
reprisk_merged

In [19]:
reprisk_merged['year'] = reprisk_merged['YearMonth'].dt.year

,gvkey,YearMonth,n_material,n_nonmaterial,n_car_1_material,n_car_5_material,n_reach_material,n_severity_material,cusip,SICS Codified Industry,Codified SICS Sector,year,industry_n_material,industry_n_car_1_material,industry_n_car_5_material,industry_n_reach_material,industry_n_severity_material
0,1004.0,2007-01,0,0,0,0,0,0,000361105,Industrial Machinery & Goods,Resource Transformation,2007,1,0,0,5,5
1,1004.0,2007-02,0,0,0,0,0,0,000361105,Industrial Machinery & Goods,Resource Transformation,2007,0,2,2,7,7
2,1004.0,2007-03,0,0,0,0,0,0,000361105,Industrial Machinery & Goods,Resource Transformation,2007,1,1,1,6,6
3,1004.0,2007-04,0,0,0,0,0,0,000361105,Industrial Machinery & Goods,Resource Transformation,2007,1,0,0,7,7
4,1004.0,2007-05,0,0,0,0,0,0,000361105,Industrial Machinery & Goods,Resource Transformation,2007,0,1,1,8,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10643287,367496.0,2023-08,0,0,0,0,0,0,NaN,Real Estate,Infrastructure,2023,3,3,10,10,10
10643288,367496.0,2023-09,0,0,0,0,0,0,NaN,Real Estate,Infrastructure,2023,2,6,17,17,17
10643289,367496.0,2023-10,0,0,0,0,0,0,NaN,Real Estate,Infrastructure,2023,3,8,16,16,16
10643290,367496.0,2023-11,0,0,0,0,0,0,NaN,Real Estate,Infrastructure,2023,3,3,23,23,23


In [20]:
agg_cols = [
    'n_material',
    'n_car_1_material',
    'n_car_5_material',
    'n_reach_material',
    'n_severity_material',
]

industry_col = 'SICS Codified Industry '   # use your exact column name
ym_col       = 'YearMonth'                 # can be string or Period[M]

# 1) Industry-level monthly counts (sum across firms)
industry_counts = (
    reprisk_merged.groupby([industry_col, ym_col], as_index=False)[agg_cols]
      .sum(min_count=1)   # keeps NaN if an entire group/col is NaN
      .rename(columns={c: f'industry_{c}' for c in agg_cols})
)

# 2) (Optional) merge back so each firm-month has the industry counts attached
df_with_industry = reprisk_merged.merge(industry_counts, on=[industry_col, ym_col], how='left')
df_with_industry

,gvkey,YearMonth,n_material,n_nonmaterial,n_car_1_material,n_car_5_material,n_reach_material,n_severity_material,cusip,SICS Codified Industry,Codified SICS Sector,year,industry_n_material,industry_n_car_1_material,industry_n_car_5_material,industry_n_reach_material,industry_n_severity_material
0,1004.0,2007-01,0,0,0,0,0,0,000361105,Industrial Machinery & Goods,Resource Transformation,2007,1,0,0,5,5
1,1004.0,2007-02,0,0,0,0,0,0,000361105,Industrial Machinery & Goods,Resource Transformation,2007,0,2,2,7,7
2,1004.0,2007-03,0,0,0,0,0,0,000361105,Industrial Machinery & Goods,Resource Transformation,2007,1,1,1,6,6
3,1004.0,2007-04,0,0,0,0,0,0,000361105,Industrial Machinery & Goods,Resource Transformation,2007,1,0,0,7,7
4,1004.0,2007-05,0,0,0,0,0,0,000361105,Industrial Machinery & Goods,Resource Transformation,2007,0,1,1,8,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10543939,356128.0,2023-08,0,0,0,0,0,0,48581R205,Consumer Finance,Financials,2023,16,3,3,14,14
10543940,356128.0,2023-09,0,0,0,0,0,0,48581R205,Consumer Finance,Financials,2023,12,0,0,10,10
10543941,356128.0,2023-10,0,0,0,0,0,0,48581R205,Consumer Finance,Financials,2023,8,0,0,7,7
10543942,356128.0,2023-11,0,0,0,0,0,0,48581R205,Consumer Finance,Financials,2023,3,1,0,1,1


In [23]:
df_with_industry.dropna(subset=['cusip'], inplace=True)
df_with_industry

In [24]:
def fetch_ff3_mom_monthly(db, start_date, end_date):
    sql = f"""
        SELECT date, mktrf, smb, hml, rf, umd
        FROM ff.factors_monthly
        WHERE date BETWEEN '{start_date}' AND '{end_date}'
    """
    ff = db.raw_sql(sql, date_cols=['date']).sort_values('date')
    for c in ['mktrf','smb','hml','rf','umd']:
        ff[c] = pd.to_numeric(ff[c], errors='coerce') / 100.0  # percent -> decimal
    return ff

,date,mktrf,smb,hml,rf,umd
0,2007-01-01,0.000138,0.000009,-0.000068,0.000044,0.000017
1,2007-02-01,-0.000196,0.000123,-0.000008,0.000038,-0.000129
2,2007-03-01,0.000071,0.000009,-0.000088,0.000043,0.000258
3,2007-04-01,0.000349,-0.000221,-0.000144,0.000044,-0.000016
4,2007-05-01,0.000323,0.000016,-0.000059,0.000041,-0.000033
...,...,...,...,...,...,...
199,2023-08-01,-0.000236,-0.000314,-0.000115,0.000045,0.000378
200,2023-09-01,-0.000523,-0.000251,0.000149,0.000043,0.000033
201,2023-10-01,-0.000315,-0.000386,0.00002,0.000047,0.000167
202,2023-11-01,0.000888,-0.00001,0.000167,0.000044,0.000256


In [25]:
# --- merge to your monthly df (dfm) ---
df_with_industry['YearMonth'] = pd.to_datetime(df_with_industry['YearMonth'].astype(str), errors='coerce')
start_date = df_with_industry['YearMonth'].min().replace(day=1).strftime('%Y-%m-%d')
end_date   = df_with_industry['YearMonth'].max().replace(day=31).strftime('%Y-%m-%d')

fac = fetch_ff3_mom_monthly(db, start_date, end_date)
fac

,gvkey,YearMonth,n_material,n_nonmaterial,n_car_1_material,n_car_5_material,n_reach_material,n_severity_material,cusip,SICS Codified Industry,...,industry_n_car_1_material,industry_n_car_5_material,industry_n_reach_material,industry_n_severity_material,date,mktrf,smb,hml,rf,umd
0,1004.0,2007-01-01,0,0,0,0,0,0,000361105,Industrial Machinery & Goods,...,0,0,5,5,2007-01-01,0.000138,0.000009,-0.000068,0.000044,0.000017
1,1004.0,2007-02-01,0,0,0,0,0,0,000361105,Industrial Machinery & Goods,...,2,2,7,7,2007-02-01,-0.000196,0.000123,-0.000008,0.000038,-0.000129
2,1004.0,2007-03-01,0,0,0,0,0,0,000361105,Industrial Machinery & Goods,...,1,1,6,6,2007-03-01,0.000071,0.000009,-0.000088,0.000043,0.000258
3,1004.0,2007-04-01,0,0,0,0,0,0,000361105,Industrial Machinery & Goods,...,0,0,7,7,2007-04-01,0.000349,-0.000221,-0.000144,0.000044,-0.000016
4,1004.0,2007-05-01,0,0,0,0,0,0,000361105,Industrial Machinery & Goods,...,1,1,8,9,2007-05-01,0.000323,0.000016,-0.000059,0.000041,-0.000033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2390467,356128.0,2023-08-01,0,0,0,0,0,0,48581R205,Consumer Finance,...,3,3,14,14,2023-08-01,-0.000236,-0.000314,-0.000115,0.000045,0.000378
2390468,356128.0,2023-09-01,0,0,0,0,0,0,48581R205,Consumer Finance,...,0,0,10,10,2023-09-01,-0.000523,-0.000251,0.000149,0.000043,0.000033
2390469,356128.0,2023-10-01,0,0,0,0,0,0,48581R205,Consumer Finance,...,0,0,7,7,2023-10-01,-0.000315,-0.000386,0.00002,0.000047,0.000167
2390470,356128.0,2023-11-01,0,0,0,0,0,0,48581R205,Consumer Finance,...,1,0,1,1,2023-11-01,0.000888,-0.00001,0.000167,0.000044,0.000256


In [26]:
dfm = df_with_industry.merge(
    fac[['date','mktrf','smb','hml','rf','umd']],
    left_on='YearMonth', right_on='date', how='left'
)
dfm

In [ ]:
df_with_industry.to_csv('data/incidents_rolling_industry.csv', index=False)